# Philadelphia: a city about to boom?

**Michael Uftring**   
**Indiana University**   
**I590 Data Visualization**   
**Fall, 2017**   


what is the purpose of this notebook?

The Change in Unemployment Rate over time with Unemployment Rate trend visualization creates a basic framework for assessment and comparison of cities. This approach will be used to establish the “Gold Standard” cities: those with the best numbers and trends, and perhaps sprinkle in less tangible qualities like general perception of being a “booming city.”



Establish the *Gold Standard* cities:
- Change in Unemployment Rate over time with Unemployment Rate trend visualization
- Industry and Occupation trends in these cities: what’s growing? what’s shrinking?


ToDo:

- load Top 100 cities
- extract list of cities
- generate BLS request for measureList = ["unemployment-rate"]


In [1]:
import requests
import json
import prettytable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import bls

%matplotlib inline

In [2]:
bls.init()

In [3]:
cities = pd.read_csv("us-cities-top100-cleaner.csv")

In [4]:
cities.head()

,Unnamed: 0,rank,city,statename,population_2016,population_2010,latitide,longitude,state
0,0,1,New York,New York,"8,537,673","8,175,133",﻿40.6635,-73.9387,NY
1,1,2,Los Angeles,California,"3,976,322","3,792,621",﻿34.0194,-118.4108,CA
2,2,3,Chicago,Illinois,"2,704,958","2,695,598",﻿41.8376,-87.6818,IL
3,3,4,Houston,Texas,"2,303,482","2,100,263",﻿29.7866,-95.3909,TX
4,4,5,Phoenix,Arizona,"1,615,017","1,445,632",﻿33.5722,-112.0901,AZ


In [125]:
cities

,Unnamed: 0,rank,city,statename,population_2016,population_2010,latitide,longitude,state
0,0,1,New York,New York,"8,537,673","8,175,133",﻿40.6635,-73.9387,NY
1,1,2,Los Angeles,California,"3,976,322","3,792,621",﻿34.0194,-118.4108,CA
2,2,3,Chicago,Illinois,"2,704,958","2,695,598",﻿41.8376,-87.6818,IL
3,3,4,Houston,Texas,"2,303,482","2,100,263",﻿29.7866,-95.3909,TX
4,4,5,Phoenix,Arizona,"1,615,017","1,445,632",﻿33.5722,-112.0901,AZ
5,5,6,Philadelphia,Pennsylvania,"1,567,872","1,526,006",﻿40.0094,-75.1333,PA
6,6,7,San Antonio,Texas,"1,492,510","1,327,407",﻿29.4724,-98.5251,TX
7,7,8,San Diego,California,"1,406,630","1,307,402",﻿32.8153,-117.1350,CA
8,8,9,Dallas,Texas,"1,317,929","1,197,816",﻿32.7933,-96.7665,TX
9,9,10,San Jose,California,"1,025,350","945,942",﻿37.2967,-121.8189,CA


In [155]:
cityName = 'St. Louis'
bls.findCityTownCode(cityName)

['CT2757220000000', 'CT2965000000000']

In [5]:
cityList = cities.city.values

In [74]:
startYear = 2008
endYear = 2017

dataSet = "LA"

seasonalAdjustment = "no"
if seasonalAdjustment == "yes":
    saCode = "S"
else:
    saCode = "U"

availableMeasures = ["labor-force", "employment", "unemployment", "unemployment-rate"]
measureList = ["unemployment-rate"]

colNames = ["seriesid","year","month","period","date","state","city","measure","value","footnotes"]

build and send requests, 10 cities at a time...

In [69]:
def buildSeriesIDs(cityList):
    seriesIDs = []
    for city in cityList:
        cityCode = bls.findCityTownCode(city)
        if len(cityCode) > 0:
            for measure in measureList:
                measureCode = bls.findMeasureCode(measure)
                seriesID = "{}{}{}{}".format(dataSet, saCode, cityCode[0], measureCode)
                #print("city: {} code: {} measure: {} code: {} seriesID: {}".format(city, cityCode, measure, measureCode, seriesID))
                seriesIDs.append(seriesID)
        else:
            print("Did not find City Code for {}".format(city))
    return seriesIDs

In [72]:
headers = {'Content-type': 'application/json'}
responses = []

count = 0
requestNumber = 0
segmentSize = 10
cityList2 = []
for city in cityList:
    count += 1
    cityList2.append(city)
    if count % segmentSize == 0:
        requestNumber += 1
        # construct Series IDs
        seriesIDs = buildSeriesIDs(cityList2)
        # build the request
        request = json.dumps({"seriesid": seriesIDs,"startyear":"{}".format(startYear), "endyear":"{}".format(endYear)})    
        # get the data using API
        print("Sending request {} ...".format(requestNumber))
        p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=request, headers=headers)
        # extract and save the response
        json_data = json.loads(p.text)
        responses.append(json_data)        
        cityList2 = []


Sending request 1 ...
Sending request 2 ...
Sending request 3 ...
Sending request 4 ...
Sending request 5 ...
Sending request 6 ...
Sending request 7 ...
Sending request 8 ...
Sending request 9 ...
Sending request 10 ...


In [73]:
len(responses)

10

In [75]:
data = []
for json_data in responses:
    for series in json_data['Results']['series']:
        x=prettytable.PrettyTable(colNames)
        seriesId = series['seriesID']
        for item in series['data']:
            year = item['year']
            period = item['period']
            month = bls.code2month(period)
            date = bls.dateFrom(year, month)
            state = bls.decodeState(seriesId)
            city = bls.decodeCityTown(seriesId)
            measure = bls.decodeMeasure(seriesId)
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','
            if 'M01' <= period <= 'M12':
                row = [seriesId,year,month,period,date,state,city,measure,value,footnotes[0:-1]]
                x.add_row(row)
                data.append(row)

In [76]:
len(data)

11677

In [77]:
df = pd.DataFrame(data, columns=colNames)

In [78]:
len(df.city.unique())

100

In [136]:
# national rates are from here:
#   https://data.bls.gov/timeseries/LNS14000000

# these are the July values for each year
nationalRates = {
    "2007": 4.7,
    "2008": 5.8,
    "2009": 9.5,
    "2010": 9.4,
    "2011": 9.0,
    "2012": 8.2,
    "2013": 7.3,
    "2014": 6.2,
    "2015": 5.2,
    "2016": 4.9,
    "2017": 4.3
}

In [138]:
year = "2010"
nationalRates[year]

9.4

In [139]:
series = []
for city in df.city.unique():
    cd = df[(df.city==city) & (df.month=='July')].sort_values('year').copy()
    cd.drop(['seriesid','month','period','date','footnotes'], axis=1, inplace=True)
    beginYear = cd.head(1).year.values[0]
    beginValue = float(cd.head(1).value.values[0])
    beginCTN = beginValue - float(nationalRates[beginYear])
    endYear = cd.tail(1).year.values[0]
    endValue = float(cd.tail(1).value.values[0])
    endCTN = endValue - float(nationalRates[endYear])
    delta = endValue - beginValue
    series.append([city,beginYear,beginValue, beginCTN,endYear,endValue,endCTN,delta])

In [140]:
len(series)

100

In [141]:
series

[['New York city',
  '2008',
  5.7,
  -0.09999999999999964,
  '2017',
  5.0,
  0.7000000000000002,
  -0.7000000000000002],
 ['Philadelphia County/city',
  '2008',
  7.4,
  1.6000000000000005,
  '2017',
  6.9,
  2.6000000000000005,
  -0.5],
 ['Phoenix city',
  '2008',
  6.7,
  0.9000000000000004,
  '2017',
  4.7,
  0.40000000000000036,
  -2.0],
 ['San Jose city',
  '2008',
  7.1,
  1.2999999999999998,
  '2017',
  4.2,
  -0.09999999999999964,
  -2.8999999999999995],
 ['Houston city',
  '2008',
  5.0,
  -0.7999999999999998,
  '2017',
  4.7,
  0.40000000000000036,
  -0.2999999999999998],
 ['San Diego city',
  '2008',
  6.4,
  0.6000000000000005,
  '2017',
  4.4,
  0.10000000000000053,
  -2.0],
 ['San Antonio city',
  '2008',
  4.9,
  -0.8999999999999995,
  '2017',
  3.7,
  -0.5999999999999996,
  -1.2000000000000002],
 ['Dallas city',
  '2008',
  5.7,
  -0.09999999999999964,
  '2017',
  3.9,
  -0.3999999999999999,
  -1.8000000000000003],
 ['Chicago Heights city', '2008', 10.5, 4.7, '2017', 

In [142]:
dfs = pd.DataFrame(series, columns=['city','begin_year','begin_value','begin_ctn','end_year','end_value','end_ctn','delta_value'])

In [143]:
dfs.head()

,city,begin_year,begin_value,begin_ctn,end_year,end_value,end_ctn,delta_value
0,New York city,2008,5.7,-0.1,2017,5.0,0.7,-0.7
1,Philadelphia County/city,2008,7.4,1.6,2017,6.9,2.6,-0.5
2,Phoenix city,2008,6.7,0.9,2017,4.7,0.4,-2.0
3,San Jose city,2008,7.1,1.3,2017,4.2,-0.1,-2.9
4,Houston city,2008,5.0,-0.8,2017,4.7,0.4,-0.3


In [144]:
dfm = pd.melt(dfs,id_vars=['city'], value_vars=['begin_year','begin_value','begin_ctn','end_year','end_value','end_ctn','delta_value'])

In [145]:
dfm[dfm.variable=='begin_year'].value.unique()

array(['2008', '2010'], dtype=object)

interesting, so there's one or more cities for which the data begins in 2010 and not 2008

In [146]:
dfm[(dfm.variable=='begin_year') & (dfm.value=='2010')]

,city,variable,value
24,Washington township (Macomb County),begin_year,2010


In [147]:
dfm.head()

,city,variable,value
0,New York city,begin_year,2008
1,Philadelphia County/city,begin_year,2008
2,Phoenix city,begin_year,2008
3,San Jose city,begin_year,2008
4,Houston city,begin_year,2008


In [148]:
dfm[dfm.variable=='delta_value'].sort_values('value')

,city,variable,value
624,Washington township (Macomb County),delta_value,-8.8
620,Detroit city,delta_value,-7.6
663,Stockton city,delta_value,-4.7
655,Santa Ana city,delta_value,-4.4
696,San Bernardino city,delta_value,-4.3
652,Cleveland city,delta_value,-3.7
609,Los Angeles city,delta_value,-3.5
677,Fort Wayne city,delta_value,-3.4
685,Madison Heights city,delta_value,-3.3
633,Sacramento city,delta_value,-3.2


In [ ]:
dfm[dfm.variable=='delta_value'].sort_values('value')

In [149]:
cityName = 'Detroit city'
month='July'
df[(df.city==cityName) & (df.month==month)].sort_values('year')

,seriesid,year,month,period,date,state,city,measure,value,footnotes
2451,LAUCT262200000000003,2008,July,M07,2008/07,Michigan,Detroit city,unemployment-rate,17.2,
2439,LAUCT262200000000003,2009,July,M07,2009/07,Michigan,Detroit city,unemployment-rate,28.3,
2427,LAUCT262200000000003,2010,July,M07,2010/07,Michigan,Detroit city,unemployment-rate,26.8,
2415,LAUCT262200000000003,2011,July,M07,2011/07,Michigan,Detroit city,unemployment-rate,24.2,
2403,LAUCT262200000000003,2012,July,M07,2012/07,Michigan,Detroit city,unemployment-rate,22.2,
2391,LAUCT262200000000003,2013,July,M07,2013/07,Michigan,Detroit city,unemployment-rate,21.7,
2379,LAUCT262200000000003,2014,July,M07,2014/07,Michigan,Detroit city,unemployment-rate,18.5,
2367,LAUCT262200000000003,2015,July,M07,2015/07,Michigan,Detroit city,unemployment-rate,13.8,
2355,LAUCT262200000000003,2016,July,M07,2016/07,Michigan,Detroit city,unemployment-rate,12.9,
2343,LAUCT262200000000003,2017,July,M07,2017/07,Michigan,Detroit city,unemployment-rate,9.6,


In [150]:
cityName = 'Detroit city'
dfs[dfs.city==cityName]

,city,begin_year,begin_value,begin_ctn,end_year,end_value,end_ctn,delta_value
20,Detroit city,2008,17.2,11.4,2017,9.6,5.3,-7.6


In [151]:
dfs.sort_values('delta_value')

,city,begin_year,begin_value,begin_ctn,end_year,end_value,end_ctn,delta_value
24,Washington township (Macomb County),2010,12.3,2.9,2017,3.5,-0.8,-8.8
20,Detroit city,2008,17.2,11.4,2017,9.6,5.3,-7.6
63,Stockton city,2008,12.9,7.1,2017,8.2,3.9,-4.7
55,Santa Ana city,2008,9.2,3.4,2017,4.8,0.5,-4.4
96,San Bernardino city,2008,11.8,6.0,2017,7.5,3.2,-4.3
52,Cleveland city,2008,7.8,2.0,2017,4.1,-0.2,-3.7
9,Los Angeles city,2008,9.1,3.3,2017,5.6,1.3,-3.5
77,Fort Wayne city,2008,6.8,1.0,2017,3.4,-0.9,-3.4
85,Madison Heights city,2008,8.1,2.3,2017,4.8,0.5,-3.3
33,Sacramento city,2008,8.8,3.0,2017,5.6,1.3,-3.2


In [153]:
# best overall delta (since before the recession), and better than the current national rate
dfs[dfs.end_ctn <= 0.0].sort_values('delta_value')

,city,begin_year,begin_value,begin_ctn,end_year,end_value,end_ctn,delta_value
24,Washington township (Macomb County),2010,12.3,2.9,2017,3.5,-0.8,-8.8
52,Cleveland city,2008,7.8,2.0,2017,4.1,-0.2,-3.7
77,Fort Wayne city,2008,6.8,1.0,2017,3.4,-0.9,-3.4
58,Tampa city,2008,7.3,1.5,2017,4.2,-0.1,-3.1
18,Denver County/city,2008,5.4,-0.4,2017,2.4,-1.9,-3.0
79,St. Petersburg city,2008,6.8,1.0,2017,3.8,-0.5,-3.0
3,San Jose city,2008,7.1,1.3,2017,4.2,-0.1,-2.9
39,Colorado Springs city,2008,5.7,-0.1,2017,2.9,-1.4,-2.8
25,Nashville-Davidson (consolidated) city,2008,5.7,-0.1,2017,3.0,-1.3,-2.7
90,Boise City city,2008,5.1,-0.7,2017,2.5,-1.8,-2.6


In [154]:
# best compared to the current national rate
dfs[dfs.end_ctn <= 0.0].sort_values('end_ctn')

,city,begin_year,begin_value,begin_ctn,end_year,end_value,end_ctn,delta_value
50,Honolulu County/city,2008,4.1,-1.7,2017,2.3,-2.0,-1.8
18,Denver County/city,2008,5.4,-0.4,2017,2.4,-1.9,-3.0
90,Boise City city,2008,5.1,-0.7,2017,2.5,-1.8,-2.6
78,Lincoln city,2008,3.2,-2.6,2017,2.7,-1.6,-0.5
39,Colorado Springs city,2008,5.7,-0.1,2017,2.9,-1.4,-2.8
92,Fremont city,2008,3.5,-2.3,2017,2.9,-1.4,-0.6
60,St. Louis Park city,2008,4.8,-1.0,2017,3.0,-1.3,-1.8
15,Austin city,2008,4.4,-1.4,2017,3.0,-1.3,-1.4
25,Nashville-Davidson (consolidated) city,2008,5.7,-0.1,2017,3.0,-1.3,-2.7
41,Arlington town,2008,4.2,-1.6,2017,3.0,-1.3,-1.2


Cities fixed:
- Washington, DC
- El Paso, TX
- St. Louis, MO
- Lexington, KY

Not sure:
- Long Beach, CA

ToDo:

- figure out how to verify city-town codes returned from `findCityTownCode` match the Top 100 City data values